# A multi-layer feature fusion method for few-shot image classification

`Credit:` This code was initially based on the implementation from [https://github.com/cnielly/prototypical-networks-omniglot](https://github.com/cnielly/prototypical-networks-omniglot.git).

## Import Packages

In [1]:
import os
import numpy as np
import cv2
import imutils
import multiprocessing as mp

import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

#Check GPU support, please do activate GPU
print(torch.cuda.is_available())

True


## Read dataset

In [4]:
def read_classes(class_path, class_name,nH): 
    datax = []
    datay = []
    images = os.listdir(class_path)
    for img in images:
        image = cv2.resize(cv2.imread(class_path + '/' + img),(nH,nH))

        # Rotate images to create new classes
        rotated_90 = imutils.rotate(image, angle=90)
        rotated_180 = imutils.rotate(image, angle=180)
        rotated_270 = imutils.rotate(image, angle=270)

        datax.extend((image, rotated_90, rotated_180, rotated_270))
        datay.extend((
            class_name + '_0',
            class_name + '_90',
            class_name + '_180',
            class_name + '_270'
        ))
        
    return np.array(datax), np.array(datay)

In [5]:
def read_images(base_directory, nH):
    """
    Reads all the classes from the base_directory
    Uses multithreading to decrease the reading time
    """
    datax = None
    datay = None
    
    pool = mp.Pool(mp.cpu_count())
    results = [pool.apply(read_classes, args=(base_directory + '/' + directory + '/', directory, nH,
                          )) for directory in os.listdir(base_directory)]
    pool.close()

    for result in results:
        if datax is None:
            datax = result[0]
            datay = result[1]
        else:
            datax = np.vstack([datax, result[0]])
            datay = np.concatenate([datay, result[1]])
    return datax, datay

**Define image size**

In [6]:
nH = 96 # image.shape = [nH,nH,3]

**Read data**

In [7]:
path_train = '/content/images/source'
path_test = '/content/images/target'

In [8]:
%%time 
trainx, trainy = read_images(path_train,nH)

CPU times: user 58.9 ms, sys: 74.2 ms, total: 133 ms
Wall time: 1.46 s


In [9]:
%%time 
testx, testy = read_images(path_test, nH)

CPU times: user 74.5 ms, sys: 78.7 ms, total: 153 ms
Wall time: 1.56 s


In [10]:
trainx.shape, trainy.shape, testx.shape, testy.shape

((840, 96, 96, 3), (840,), (840, 96, 96, 3), (840,))

## Create samples

In [11]:
def extract_sample(n_way, n_support, n_query, datax, datay):
  """
  Picks random sample of size n_support+n_querry, for n_way classes
  Args:
      n_way (int): number of classes in a classification task
      n_support (int): number of labeled examples per class in the support set
      n_query (int): number of labeled examples per class in the query set
      datax (np.array): dataset of images
      datay (np.array): dataset of labels
  Returns:
      (dict) of:
        (torch.Tensor): sample of images. Size (n_way, n_support+n_query, (dim))
        (int): n_way
        (int): n_support
        (int): n_query
  """
  sample = []
  K = np.random.choice(np.unique(datay), n_way, replace=False)
  for cls in K:
    datax_cls = datax[datay == cls]
    perm = np.random.permutation(datax_cls)
    sample_cls = perm[:(n_support+n_query)]
    sample.append(sample_cls)
  sample = np.array(sample)
  sample = torch.from_numpy(sample).float()
  sample = sample.permute(0,1,4,2,3)
  return({
      'images': sample,
      'n_way': n_way,
      'n_support': n_support,
      'n_query': n_query
      })

## Model

In [21]:
class Flatten(nn.Module):
  def __init__(self):
    super(Flatten, self).__init__()

  def forward(self, x):
    return x.view(x.size(0), -1)

def load_protonet_conv(**kwargs):
  """
  Returns:
      Model (Class ProtoNet)
  """
  x_dim = kwargs['x_dim']
  hid_dim = kwargs['hid_dim']
  z_dim = kwargs['z_dim']
  selected_model = kwargs['selected_model']

  # Conv_blocks
  if selected_model == 'conv_blocks':
      def conv_block(in_channels, out_channels):
          return nn.Sequential(
              nn.Conv2d(in_channels, out_channels, 3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(),
              nn.MaxPool2d(3)
              )
            
      encoder = nn.Sequential(
              conv_block(x_dim[0], hid_dim[0]),
              conv_block(hid_dim[0], hid_dim[1]),
              conv_block(hid_dim[1], hid_dim[2]),
              conv_block(hid_dim[2], z_dim),
              Flatten()
      )

  # ResNet50
  elif selected_model == 'resnet50':
      resnet = models.resnet50(pretrained=True)
      encoder = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, resnet.layer1, 
                              resnet.layer2, resnet.layer3, resnet.layer4, resnet.avgpool, Flatten())

      # Fix blocks
      for p in encoder[0].parameters():
          p.requires_grad = False
      for p in encoder[1].parameters():
          p.requires_grad = False

      def set_bn_fix(m):
          classname = m.__class__.__name__
          if classname.find('BatchNorm') != -1:
              for p in m.parameters(): p.requires_grad = False

      encoder.apply(set_bn_fix)
      
  # Mobilenet_v2
  elif selected_model == 'mobilenetv2':
      select_model = models.mobilenet_v2(pretrained=True)
      encoder = nn.Sequential(select_model.features, 
                            nn.AdaptiveAvgPool2d((1,1)),
                            Flatten())

  # VGG16
  elif selected_model == 'vgg16':
      vgg = models.vgg16_bn(pretrained=True)
      encoder =  nn.Sequential(vgg.features,
                              Flatten()
                              ) 

  return ProtoNet(encoder)

## Encoder

In [28]:
class ProtoNet(nn.Module):
  def __init__(self, encoder):
    """
    Args:
        encoder : CNN encoding the images in sample
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
    """
    super(ProtoNet, self).__init__()
    # Conv_blocks
    self.encoder1 = nn.Sequential(*list(encoder.children())[:1],
                        nn.MaxPool2d(5),
                        nn.MaxPool2d(5),
                        Flatten()
                        ).cuda()
    self.encoder2 = nn.Sequential(*list(encoder.children())[:2],
                        nn.MaxPool2d(3),
                        nn.MaxPool2d(2),
                        Flatten()
                        ).cuda()
    self.encoder3 = nn.Sequential(*list(encoder.children())[:3],
                        nn.MaxPool2d(2),
                        Flatten()
                        ).cuda()
    self.encoder = encoder.cuda()   

  def set_forward_loss(self, sample, selected_model: str):
    """
    Computes loss, accuracy and output for classification task
    Args:
        sample (torch.Tensor): shape (n_way, n_support+n_query, (dim)) 
        selected_model (str): The model chosen for feature extraction
    Returns:
        torch.Tensor: shape(2), loss, accuracy and y_hat
    """
    sample_images = sample['images'].cuda()
    n_way = sample['n_way']
    n_support = sample['n_support']
    n_query = sample['n_query']

    x_support = sample_images[:, :n_support]
    x_query = sample_images[:, n_support:]
   
    # Target indices are 0 ... n_way-1
    target_inds = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, n_query, 1).long()
    target_inds = Variable(target_inds, requires_grad=False)
    target_inds = target_inds.cuda()
   
    # Encode images of the support and the query set
    x = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_query.contiguous().view(n_way * n_query, *x_query.size()[2:])], 0)

    ##
    if selected_model == 'conv_blocks':
      f1 = self.encoder1.forward(x)
      f2 = self.encoder2.forward(x)
      f3 = self.encoder3.forward(x)
      f4 = self.encoder.forward(x)
      z = torch.stack((f1,
                        f2,
                        f3,
                        f4), 1).mean(1) 

    else:
      z = self.encoder.forward(x) 
    ##

    z_dim = z.size(-1)

    # Z_PROTO
    z_proto = z[:n_way*n_support]

    # Z_QUERY
    z_query = z[n_way*n_support:]

    # Compute Squared Euclidean distance
    dists = euclidean_dist(z_query, z_proto.view(n_way, n_support, z_dim).mean(1))
    
    # Compute probabilities, loss, y_hat, and accuracy
    log_p_y = F.log_softmax(-dists, dim=1).view(n_way, n_query, -1)
    loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hat = log_p_y.max(2)
    acc_val = torch.eq(y_hat, target_inds.squeeze()).float().mean()
   
    return loss_val, {
        'loss': loss_val.item(),
        'acc': acc_val.item(),
        'y_hat': y_hat
        }

## Compute similarity
Squared Euclidean distance

In [14]:
def euclidean_dist(x, y):
  """
  Args:
      x (torch.Tensor): shape (n, d). n usually n_way*n_query
      y (torch.Tensor): shape (m, d). m usually n_way
  Returns:
      torch.Tensor: shape(n, m). For each query, the euclidean distance to each centroid
  """
  n = x.size(0)
  m = y.size(0)
  d = x.size(1)
  assert d == y.size(1)

  x = x.unsqueeze(1).expand(n, m, d)
  y = y.unsqueeze(0).expand(n, m, d)

  return torch.pow(x - y, 2).sum(2)

## Training function

In [15]:
from tqdm import tqdm_notebook
from tqdm.notebook import tnrange

In [16]:
def train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size, selected_model: str):
  """
  Trains the protonet
  Args:
      model
      optimizer
      train_x (np.array): images of training set
      train_y(np.array): labels of training set
      n_way (int): number of classes in a classification task
      n_support (int): number of labeled examples per class in the support set
      n_query (int): number of labeled examples per class in the query set
      max_epoch (int): max epochs to train on
      epoch_size (int): episodes per epoch
  """
  # divide the learning rate by 2 at each epoch, as suggested in paper
  scheduler = optim.lr_scheduler.StepLR(optimizer, 20, gamma=0.5, last_epoch=-1)
  epoch = 0 # epochs done so far
  stop = False # status to know when to stop

  while epoch < max_epoch and not stop:
    running_loss = 0.0
    running_acc = 0.0

    for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch+1)):
      sample = extract_sample(n_way, n_support, n_query, train_x, train_y)
      optimizer.zero_grad()
      loss, output = model.set_forward_loss(sample, selected_model)
      
      running_loss += output['loss']
      running_acc += output['acc']
      loss.backward()
      optimizer.step()
    epoch_loss = running_loss / epoch_size
    epoch_acc = running_acc / epoch_size
    print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch+1,epoch_loss, epoch_acc))

    epoch += 1
    scheduler.step()

## Train and Test

In [17]:
def test(model, test_x, test_y, n_way, n_support, n_query, test_episode, selected_model: str):
  running_loss = 0.0
  running_acc = 0.0
  for episode in tnrange(test_episode):
    sample = extract_sample(n_way, n_support, n_query, test_x, test_y)
    loss, output = model.set_forward_loss(sample, selected_model)
    running_loss += output['loss']
    running_acc += output['acc']
  avg_loss = running_loss / test_episode
  avg_acc = running_acc / test_episode
  print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

In [32]:
%%time
import pandas as pd

train_x = trainx
train_y = trainy
test_x = testx
test_y = testy

model = load_protonet_conv(
    x_dim=(3,nH,nH),
    hid_dim=(64,64,64),
    z_dim=64,
    selected_model='conv_blocks'
    )
      
optimizer = optim.Adam(model.parameters(), lr = 0.001)

n_way = 5
N_test = 5
n_support = 1
n_query = 5

max_epoch = 100
epoch_size = 150
test_episode = 2000

print('\nTRAINING...')
train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size, 'conv_blocks')

print('\nTESTING...')
test(model, test_x, test_y, N_test, n_support, n_query, test_episode, 'conv_blocks')


TRAINING...


Epoch 1 train:   0%|          | 0/150 [00:00<?, ?it/s]

Epoch 1 -- Loss: 1.8888 Acc: 0.5445


Epoch 2 train:   0%|          | 0/150 [00:00<?, ?it/s]

Epoch 2 -- Loss: 0.6702 Acc: 0.7133


...

Epoch 99 -- Loss: 0.0011 Acc: 1.0000


Epoch 100 train:   0%|          | 0/150 [00:00<?, ?it/s]

Epoch 100 -- Loss: 0.0009 Acc: 1.0000

TESTING...


  0%|          | 0/2000 [00:00<?, ?it/s]

Test results -- Loss: 1.5863 Acc: 0.6869
CPU times: user 10min 52s, sys: 7.29 s, total: 10min 59s
Wall time: 11min 4s
